# 1. Theoretical Framework



## 1. Objective of the index
The goal of this index is to create 

# Import Required Packages

In [1056]:
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# 2. Data Selection - Importing Data

In [1057]:
# This is for macroeconomic indicators
df_gdp_per_capita = pd.read_csv('GDP per capita (PPP).csv')
df_gdp_growth_rate = pd.read_csv('GDP growth rate.csv')
df_FCE = pd.read_csv('Final consumption expenditure.csv')
df_gov_debt = pd.read_excel('Government debt.xls')
df_inflation = pd.read_csv('Inflation annually.csv')

In [1058]:
print(df_gov_debt[2023])

0        no data
1      34.832823
2      39.336605
3     116.064891
4       2.329216
5      79.602279
6        no data
7      56.048101
8      38.984028
9     205.613846
10     48.852372
11    115.876383
12     59.116753
13    123.143218
14     45.790078
15     59.655034
16     47.124195
17     45.772094
18     77.290714
19     52.048037
20     33.426103
21    177.157815
22     20.898315
23       no data
24      32.21002
25     54.274837
26     14.123071
27     41.148784
28      7.533489
29       no data
Name: 2023, dtype: object


In [1059]:
#This is the Trade and Investment factors
df_current_account_balance = pd.read_csv("current accoung balance (Macroeconomic Indicators ).csv")
df_exports = pd.read_csv("Exports of goods and services.csv")
df_imports = pd.read_csv("Imports of goods and services.csv")

In [1060]:
# This is for Human Capital
df_labour_force =pd.read_csv('Labour Force Participant Rate.csv')
df_life_expectancy_at_birth = pd.read_csv('Life expectancy at birth.csv')
df_school_enrollment = pd.read_csv('School enrollment.csv')
df_unemployment_rate = pd.read_csv('Unemployment rate.csv')


In [1061]:
#This is for infrastructure and Technology

df_access_to_electricity = pd.read_csv('Access to electricity.csv')
df_access_to_internet = pd.read_csv('indivisual using the internet.csv')
df_mobile_subscriptions = pd.read_csv('Mobile cellular subscriptions.csv')
df_LPI_Score = pd.read_excel('LPI Score.xlsx')


In [1062]:
# This is for Institution and Political Factors
df_CPI = pd.read_excel("CPI2023_Global_Results__Trends.xlsx",skiprows=3, engine='openpyxl')
df_political_stability = pd.read_csv("Political Stability and Absence of ViolenceTerrorism Percentile Rank.csv")

# This is for WGI data set we will only be using the regulatory quality
df_regulatory_quality = pd.read_excel("wgidataset.xlsx")

In [1063]:
#This is the environmental and sustainability

df_CO2_emissions = pd.read_csv("CO2 Emission.csv")
df_CO2_emissions_per_capita = pd.read_csv("CO2 Emission per capita.csv")
df_renewable_energy_consumption = pd.read_csv("Renewable energy consumption.csv")

# 3. Imputing of Missing Datas

In [1064]:

# This is for the data cleaning process
dataframes = [df_gdp_growth_rate, df_gdp_per_capita, df_FCE, df_gov_debt, df_inflation,
              df_current_account_balance, df_exports, df_imports, df_labour_force,
              df_life_expectancy_at_birth, df_school_enrollment, df_unemployment_rate,
              df_access_to_electricity, df_access_to_internet, df_mobile_subscriptions,
              df_LPI_Score, df_CPI, df_political_stability, df_regulatory_quality,
              df_CO2_emissions, df_CO2_emissions_per_capita, df_renewable_energy_consumption]


# This is to remove the unwanted data from the dataframes
for df in dataframes:
  df.replace(["..","...", "no data"], pd.NA, inplace=True)  # or np.nan if you're using numpy



In [1065]:
# Print sum of all missing valeues
def check_missing_values():
  print("The number of missing values in each dataset:")
  print("GDP per capita (PPP):", df_gdp_per_capita.isnull().sum().sum())
  print("GDP growth rate:", df_gdp_growth_rate.isnull().sum().sum())
  print("Final consumption expenditure:", df_FCE.isnull().sum().sum())
  print("Government debt:", df_gov_debt.isnull().sum().sum())
  print("Inflation annually:", df_inflation.isnull().sum().sum())
  print("Current account balance:", df_current_account_balance.isnull().sum().sum())
  print("Exports of goods and services:", df_exports.isnull().sum().sum())
  print("Imports of goods and services:", df_imports.isnull().sum().sum())
  print("Labour Force Participant Rate:", df_labour_force.isnull().sum().sum())
  print("Life expectancy at birth:", df_life_expectancy_at_birth.isnull().sum().sum())
  print("School enrollment:", df_school_enrollment.isnull().sum().sum())
  print("Unemployment rate:", df_unemployment_rate.isnull().sum().sum())
  print("Access to electricity:", df_access_to_electricity.isnull().sum().sum())
  print("Access to internet:", df_access_to_internet.isnull().sum().sum())
  print("Mobile cellular subscriptions:", df_mobile_subscriptions.isnull().sum().sum())
  print("LPI Score:", df_LPI_Score.isnull().sum().sum())
  print("CPI:", df_CPI.isnull().sum().sum())
  print("Political Stability and Absence of ViolenceTerrorism Percentile Rank:", df_political_stability.isnull().sum().sum())
  print("WGI dataset:", df_regulatory_quality.isnull().sum().sum())
  print("CO2 Emission:", df_CO2_emissions.isnull().sum().sum())
  print("CO2 Emission per capita:", df_CO2_emissions_per_capita.isnull().sum().sum())
  print("Renewable energy consumption:", df_renewable_energy_consumption.isnull().sum().sum())
  
check_missing_values()

asian_countries_standard = [
    "Afghanistan", "Bangladesh", "Bhutan", "Brunei",
    "Cambodia", "China", "India", "Indonesia", "Japan",
    "Kazakhstan", "Kyrgyzstan", "Laos", "Malaysia", "Maldives",
    "Mongolia", "Myanmar", "Nepal", "North Korea", "Pakistan",
    "Philippines", "Singapore", "South Korea", "Sri Lanka",
    "Taiwan", "Tajikistan", "Thailand", "East Timor",
    "Turkmenistan", "Uzbekistan", "Vietnam", "Macau", "Hong Kong","Afghanistan",
    "American Samoa",
    "Australia",
    "Bangladesh",
    "Bhutan",
    "Brunei",
    "Cambodia",
    "China",
    "Fiji",
    "French Polynesia",
    "Guam",
    "Hong Kong",
    "India",
    "Indonesia",
    "Japan",
    "Kazakhstan",
    "Kiribati",
    "North Korea",
    "South Korea",
    "Kyrgyzstan",
    "Laos",
    "Macau",
    "Malaysia",
    "Maldives",
    "Marshall Islands",
    "Micronesia",
    "Mongolia",
    "Myanmar",
    "Nauru",
    "Nepal",
    "New Caledonia",
    "New Zealand",
    "Northern Mariana Islands",
    "Pakistan",
    "Palau",
    "Papua New Guinea",
    "Philippines",
    "Samoa",
    "Singapore",
    "Solomon Islands",
    "Sri Lanka",
    "Tajikistan",
    "Thailand",
    "East Timor",
    "Tonga",
    "Turkmenistan",
    "Tuvalu",
    "Uzbekistan",
    "Vanuatu",
    "Vietnam",
    "Taiwan"
] 


country_name_mapping = {
    "Brunei Darussalam": "Brunei",
    "Hong Kong SAR": "Hong Kong",
    "Hong Kong SAR, China": "Hong Kong",
    "Hong Kong, China": "Hong Kong",
    "China, People's Republic of": "China",
    "Korea, Republic of": "South Korea",
    "Korea, Rep.": "South Korea",
    "Korea, South": "South Korea",
    "Korea, North": "North Korea",
    "Korea, Dem. Rep.": "North Korea",
    "Korea, Dem. People's Rep.": "North Korea",
    "Lao PDR": "Laos",
    "Lao P.D.R.": "Laos",
    "Lao People's Democratic Republic": "Laos",
    "Taiwan Province of China": "Taiwan",
    "Taiwan, China": "Taiwan",
    "Taipei,China": "Taiwan",
    "Timor-Leste": "East Timor",
    "TC<rkiye": "Turkey",
    "Türkiye, Republic of": "Turkey",
    "Türkiye": "Turkey",
    "TÃ¼rkiye": "Turkey",  # Encoding variant
    "Russian Federation": "Russia",  # Russia is partially in Asia
    "Iran, Islamic Rep.": "Iran",
    "Yemen, Rep.": "Yemen",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Macao SAR": "Macau",
    "Macao SAR, China": "Macau",
    "West Bank and Gaza": "Palestine",
    "Palestine": "Palestine",  # Redundant, but harmless
    "Viet Nam": "Vietnam",
    "Micronesia, Fed. Sts.": "Micronesia"  # Newly added
}


print(len(asian_countries_standard))

The number of missing values in each dataset:
GDP per capita (PPP): 56
GDP growth rate: 19
Final consumption expenditure: 542
Government debt: 7
Inflation annually: 84
Current account balance: 20
Exports of goods and services: 64
Imports of goods and services: 64
Labour Force Participant Rate: 191
Life expectancy at birth: 0
School enrollment: 162
Unemployment rate: 203
Access to electricity: 9
Access to internet: 74
Mobile cellular subscriptions: 67
LPI Score: 0
CPI: 1093
Political Stability and Absence of ViolenceTerrorism Percentile Rank: 27
WGI dataset: 6756
CO2 Emission: 0
CO2 Emission per capita: 0
Renewable energy consumption: 67
83


In [1066]:
def clean_dataframe(df, value_name):
    # Clean column names
    df.columns = df.columns.map(str).str.strip()

    # Standardize country names before anything else
    df["Country Name"] = df["Country Name"].replace(country_name_mapping)
    
    # Ensure year columns are numeric
    year_cols = [str(year) for year in range(2015, 2024) if str(year) in df.columns]
    df[year_cols] = df[year_cols].apply(pd.to_numeric, errors='coerce')

    # Fill missing 2023 values with the most recent available year's value
    def get_latest_value(row):
        if pd.isna(row.get("2023")):
            for year in reversed(range(2015, 2023)):
                val = row.get(str(year))
                if pd.notna(val):
                    return val
        return row.get("2023")

    df["2023"] = df.apply(get_latest_value, axis=1)

    # Drop rows with no usable 2023 data
    df = df.dropna(subset=["2023"])

    # Keep only Country Name and 2023 column
    df = df[["Country Name", "2023"]].rename(columns={"2023": value_name})

    # Aggregate if duplicates exist
    if df.duplicated(subset=["Country Name"]).any():
        df = df.groupby("Country Name", as_index=False)[value_name].mean()

    return df


In [1067]:
df_gdp_per_capita = clean_dataframe(df_gdp_per_capita, 'gdp_per_capita_2023')
df_gdp_growth_rate = clean_dataframe(df_gdp_growth_rate, 'gdp_growth_rate_2023')
df_FCE = clean_dataframe(df_FCE, 'final_consumption_expenditure_2023')
print(len(df_gov_debt))
df_gov_debt = clean_dataframe(df_gov_debt, 'government_debt_2023')

print(len(df_gov_debt))
df_inflation = clean_dataframe(df_inflation, 'inflation_rate_2023')


30
30


In [1068]:
df_current_account_balance = clean_dataframe(df_current_account_balance, 'current_account_balance_2023')
df_exports = clean_dataframe(df_exports, 'exports_2023')
df_imports = clean_dataframe(df_imports, 'imports_2023')


In [1069]:
df_labour_force = clean_dataframe(df_labour_force, 'labour_force_participation_rate_2023')
df_life_expectancy_at_birth = clean_dataframe(df_life_expectancy_at_birth, 'life_expectancy_at_birth_2023')
df_school_enrollment = clean_dataframe(df_school_enrollment, 'school_enrollment_rate_2023')
df_unemployment_rate = clean_dataframe(df_unemployment_rate, 'unemployment_rate_2023')


In [1070]:
df_access_to_electricity = clean_dataframe(df_access_to_electricity, 'access_to_electricity_2023')
df_access_to_internet = clean_dataframe(df_access_to_internet, 'access_to_internet_2023')
df_mobile_subscriptions = clean_dataframe(df_mobile_subscriptions, 'mobile_subscriptions_2023')


In [1071]:
df_LPI_Score = df_LPI_Score[["Economy", "LPI Score"]]
df_LPI_Score.rename(columns={
    "Economy": "Country Name",
    "LPI Score": "lpi_score_2023"
}, inplace=True)

df_LPI_Score["Country Name"] = df_LPI_Score["Country Name"].replace(country_name_mapping)


C:\Users\DEcla\AppData\Local\Temp\ipykernel_36416\2607823054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LPI_Score.rename(columns={
C:\Users\DEcla\AppData\Local\Temp\ipykernel_36416\2607823054.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LPI_Score["Country Name"] = df_LPI_Score["Country Name"].replace(country_name_mapping)


In [1072]:
# CPI DataFrame – include all countries
df_CPI = df_CPI[['Country / Territory', 'CPI score 2023']].rename(columns={
    "Country / Territory": "Country Name",
    "CPI score 2023": "cpi_score_2023"
})

df_CPI["Country Name"] = df_CPI["Country Name"].replace(country_name_mapping)
# Political stability (already standardized with your clean_dataframe function)
df_political_stability = clean_dataframe(df_political_stability, 'political_stability_2023')

# Regulatory quality – no country filter applied
df_regulatory_quality = df_regulatory_quality[
    (df_regulatory_quality['indicator'] == 'rq') &
    (df_regulatory_quality['year'] > 2014) & 
    (df_regulatory_quality['year'] < 2023)
]

df_regulatory_quality = (
    df_regulatory_quality
    .groupby('countryname')['estimate']
    .mean()
    .reset_index()
    .rename(columns={'estimate': 'regulatory_quality_2023', 'countryname': 'Country Name'})
)

df_regulatory_quality["Country Name"] = df_regulatory_quality["Country Name"].replace(country_name_mapping)

In [1073]:
df_CO2_emissions = clean_dataframe(df_CO2_emissions, 'co2_emissions_2023')
df_CO2_emissions_per_capita = clean_dataframe(df_CO2_emissions_per_capita, 'co2_emissions_per_capita_2023')
df_renewable_energy_consumption = clean_dataframe(df_renewable_energy_consumption, 'renewable_energy_consumption_2023')


In [1074]:

check_missing_values()


The number of missing values in each dataset:
GDP per capita (PPP): 0
GDP growth rate: 0
Final consumption expenditure: 0
Government debt: 0
Inflation annually: 0
Current account balance: 0
Exports of goods and services: 0
Imports of goods and services: 0
Labour Force Participant Rate: 0
Life expectancy at birth: 0
School enrollment: 0
Unemployment rate: 0
Access to electricity: 0
Access to internet: 0
Mobile cellular subscriptions: 0
LPI Score: 0
CPI: 0
Political Stability and Absence of ViolenceTerrorism Percentile Rank: 0
WGI dataset: 1
CO2 Emission: 0
CO2 Emission per capita: 0
Renewable energy consumption: 0


In [1075]:
merged_df = df_gdp_per_capita.copy()

database_to_merge = [
    df_gdp_growth_rate, df_FCE, df_gov_debt, df_inflation,
    df_current_account_balance, df_exports, df_imports,
    df_labour_force, df_life_expectancy_at_birth, df_school_enrollment,
    df_unemployment_rate, df_access_to_electricity, df_access_to_internet,
    df_mobile_subscriptions, df_LPI_Score, df_CPI, df_political_stability,
    df_regulatory_quality, df_CO2_emissions, df_CO2_emissions_per_capita,
    df_renewable_energy_consumption
]

for df in database_to_merge:
    merged_df = pd.merge(merged_df, df, on='Country Name', how='outer')
    
print("The number of rows in the merged dataframe:", merged_df.shape[0])
print("The number of columns in the merged dataframe:", merged_df.shape[1])

print(merged_df.head())

The number of rows in the merged dataframe: 230
The number of columns in the merged dataframe: 23
  Country Name  gdp_per_capita_2023  gdp_growth_rate_2023  \
0  Afghanistan          2211.280635              2.710887   
1    Australia         70497.152450              3.441992   
2   Bangladesh          9147.777507              5.775112   
3       Bhutan         15063.810570              5.213868   
4       Brunei         85267.553050              1.407068   

   final_consumption_expenditure_2023  government_debt_2023  \
0                           53.269473              7.797235   
1                           36.084082             34.832823   
2                           35.069191             39.336605   
3                           49.823243            116.064891   
4                           44.481585              2.329216   

   inflation_rate_2023  current_account_balance_2023  exports_2023  \
0            -4.644709                    -15.718303     16.775136   
1             5.

In [1076]:
# Step 1: Apply mapping to standardize country names
merged_df["Country Name"] = merged_df["Country Name"].replace(country_name_mapping)

# Step 2: Filter to Asian countries only
merged_df = merged_df[merged_df["Country Name"].isin(asian_countries_standard)]

# Optional: reset index if needed
merged_df.reset_index(drop=True, inplace=True)

# Final check
print("Filtered Asian countries dataframe:")
print("Rows:", merged_df.shape[0])
print("Columns:", merged_df.shape[1])
print(merged_df.head())

Filtered Asian countries dataframe:
Rows: 51
Columns: 23
  Country Name  gdp_per_capita_2023  gdp_growth_rate_2023  \
0  Afghanistan          2211.280635              2.710887   
1    Australia         70497.152450              3.441992   
2   Bangladesh          9147.777507              5.775112   
3       Bhutan         15063.810570              5.213868   
4       Brunei         85267.553050              1.407068   

   final_consumption_expenditure_2023  government_debt_2023  \
0                           53.269473              7.797235   
1                           36.084082             34.832823   
2                           35.069191             39.336605   
3                           49.823243            116.064891   
4                           44.481585              2.329216   

   inflation_rate_2023  current_account_balance_2023  exports_2023  \
0            -4.644709                    -15.718303     16.775136   
1             5.597015                     -0.300085     

In [1077]:
print(merged_df.isnull().sum().sort_values(ascending=False))

lpi_score_2023                          23
government_debt_2023                    22
cpi_score_2023                          15
school_enrollment_rate_2023             14
inflation_rate_2023                     14
current_account_balance_2023            12
gdp_per_capita_2023                     12
unemployment_rate_2023                  12
access_to_internet_2023                 10
final_consumption_expenditure_2023      10
exports_2023                            10
imports_2023                            10
mobile_subscriptions_2023                9
labour_force_participation_rate_2023     9
political_stability_2023                 9
access_to_electricity_2023               7
gdp_growth_rate_2023                     7
co2_emissions_per_capita_2023            6
co2_emissions_2023                       6
renewable_energy_consumption_2023        6
life_expectancy_at_birth_2023            6
regulatory_quality_2023                  3
Country Name                             0
dtype: int6

In [1078]:
merged_df["missing_values"] = merged_df.isnull().sum(axis=1)

missing_data_df = merged_df[merged_df["missing_values"] > 0].copy()

print("Countries with missing values:")
print(missing_data_df[["Country Name", "missing_values"]].sort_values(by="missing_values", ascending=False))



Countries with missing values:
                Country Name  missing_values
47              Turkmenistan              19
49                    Taiwan              18
44                Kazakhstan              18
48                Uzbekistan              18
45                Kyrgyzstan              18
46                Tajikistan              18
43  Northern Mariana Islands              13
39            American Samoa              12
50               North Korea              12
40          French Polynesia              10
42             New Caledonia               9
41                      Guam               9
19                Micronesia               7
36                    Tuvalu               6
18          Marshall Islands               5
22                     Nauru               4
21                   Myanmar               4
26                     Palau               4
15                     Macau               4
27          Papua New Guinea               3
12                  Kiri

In [1079]:
merged_df = merged_df[merged_df.isnull().sum(axis=1) <= 5]  # Keep rows with less than 5 NaN values
merged_df.drop(columns=["missing_values"], inplace=True)  # Drop the helper column

In [1080]:
merged_df.skew(numeric_only=True).sort_values(ascending=False)


co2_emissions_2023                      5.627267
co2_emissions_per_capita_2023           5.083421
gdp_growth_rate_2023                    4.815450
mobile_subscriptions_2023               4.118899
inflation_rate_2023                     2.149389
gdp_per_capita_2023                     2.091706
exports_2023                            2.040768
imports_2023                            1.536153
government_debt_2023                    1.452471
unemployment_rate_2023                  1.319936
final_consumption_expenditure_2023      1.074583
renewable_energy_consumption_2023       0.816311
cpi_score_2023                          0.798870
life_expectancy_at_birth_2023           0.436786
lpi_score_2023                         -0.063466
political_stability_2023               -0.416211
labour_force_participation_rate_2023   -0.483406
school_enrollment_rate_2023            -0.549100
access_to_internet_2023                -0.616857
current_account_balance_2023           -0.998415
access_to_electricit

In [1081]:
# List of outlier columns
outliers_col = [
    "co2_emissions_2023", "co2_emissions_per_capita_2023",
    "gdp_growth_rate_2023", "mobile_subscriptions_2023", "inflation_rate_2023",
    "gdp_per_capita_2023", "exports_2023", "imports_2023",
    "government_debt_2023", "unemployment_rate_2023",
    "final_consumption_expenditure_2023"
]

# Step 1: Handle outlier columns with conditional logic
for col in outliers_col:
    median_val = merged_df[col].median()
    mean_val = merged_df[col].mean()

    # Missing values in this column
    missing_mask = merged_df[col].isna()

    # Rows with missing values in any other outlier columns
    other_cols = [c for c in outliers_col if c != col]
    outlier_rows = merged_df[other_cols].isna().any(axis=1)

    # Fill using median for outlier rows
    merged_df.loc[missing_mask & outlier_rows, col] = median_val

    # Fill using mean for non-outlier rows
    merged_df.loc[missing_mask & ~outlier_rows, col] = mean_val

# Step 2: Fill all other columns (not in outliers_col) with mean
for col in merged_df.columns:
    if col not in outliers_col and merged_df[col].dtype in ['float64', 'int64']:
        mean_val = merged_df[col].mean()
        merged_df[col] = merged_df[col].fillna(mean_val)
